In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
# Load environment variables
load_dotenv()

# Initialize Groq chat
groq_chat = ChatGroq(
    temperature=0.7,
    model_name="llama-3.1-8b-instant",  # Groq's model
    api_key=os.getenv("GROQ_API_KEY")
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

# Content creation workflow
outline_prompt = ChatPromptTemplate.from_template("""
Create a detailed outline for a blog post about: {topic}
Include 4-5 main sections and 2-3 key points for each section.

Topic: {topic}
Target Audience: {audience}

Outline:
""")

draft_prompt = ChatPromptTemplate.from_template("""
Write a blog post draft based on this outline:

OUTLINE:
{outline}

TOPIC: {topic}
TARGET AUDIENCE: {audience}
TONE: {tone}

Write a comprehensive blog post draft:
""")

review_prompt = ChatPromptTemplate.from_template("""
Review this blog post draft and provide specific feedback for improvement:

BLOG DRAFT:
{draft}

Provide constructive feedback focusing on:
1. Content structure and flow
2. Clarity and readability  
3. Engagement level
4. Areas that need more detail or examples

Feedback:
""")

revision_prompt = ChatPromptTemplate.from_template("""
Based on the feedback below, revise and improve the blog post:

ORIGINAL DRAFT:
{draft}

FEEDBACK:
{feedback}

Write the improved version of the blog post:
""")

# Create individual chains
outline_chain = outline_prompt | groq_chat | StrOutputParser()
draft_chain = draft_prompt | groq_chat | StrOutputParser() 
review_chain = review_prompt | groq_chat | StrOutputParser()
revision_chain = revision_prompt | groq_chat | StrOutputParser()

# Simulate human feedback (in real app, this would come from user input)
def simulate_human_feedback(draft):
    print("\n" + "="*60)
    print("DRAFT FOR REVIEW:")
    print("="*60)
    print(draft[:500] + "..." if len(draft) > 500 else draft)
    print("\n" + "="*60)
    
    # Return simulated feedback
    return """The draft has good structure but could be improved:
1. Add more real-world examples to make it engaging
2. Simplify some technical jargon for the target audience
3. Include practical tips or actionable advice
4. Add a compelling introduction to hook readers"""

# Build the workflow step by step (more reliable than complex chains)
def content_workflow(topic, audience="general readers", tone="professional"):
    print(f"Starting content workflow for: {topic}")
    
    # Step 1: Create outline
    print("\nStep 1: Creating outline...")
    outline = outline_chain.invoke({
        "topic": topic, 
        "audience": audience
    })
    print("✓ Outline created")
    
    # Step 2: Write draft
    print("\nStep 2: Writing draft...")
    draft = draft_chain.invoke({
        "outline": outline,
        "topic": topic,
        "audience": audience, 
        "tone": tone
    })
    print("✓ Draft written")
    
    # Step 3: Get feedback
    print("\nStep 3: Getting feedback...")
    feedback = simulate_human_feedback(draft)
    print("✓ Feedback received")
    
    # Step 4: Revise
    print("\nStep 4: Revising based on feedback...")
    final_draft = revision_chain.invoke({
        "draft": draft,
        "feedback": feedback
    })
    print("✓ Revision complete")
    
    return {
        "outline": outline,
        "initial_draft": draft,
        "feedback": feedback,
        "final_draft": final_draft
    }

# Run the workflow
print("CONTENT CREATION WORKFLOW")
print("=" * 50)

workflow_result = content_workflow(
    topic="The Impact of AI on Content Marketing",
    audience="marketing professionals", 
    tone="insightful and practical"
)

# Display results
print("\n" + " FINAL RESULTS")
print("=" * 50)

print("\n OUTLINE:")
print("-" * 30)
print(workflow_result["outline"])

print("\n FEEDBACK:")
print("-" * 30) 
print(workflow_result["feedback"])

print("\n FINAL DRAFT:")
print("=" * 50)
print(workflow_result["final_draft"])



In [ ]:
# Minimal working version of the simple chain
def minimal_content_chain(topic, audience, tone):
    """Minimal version that definitely works"""
    print(f"\n Creating content for: {topic}")
    
    # Step 1: Create outline
    outline = outline_chain.invoke({"topic": topic, "audience": audience})
    print("✓ Outline created")
    
    # Step 2: Create draft
    draft = draft_chain.invoke({
        "outline": outline, 
        "topic": topic, 
        "audience": audience, 
        "tone": tone
    })
    print("✓ Draft created")
    
    return {"outline": outline, "draft": draft}

# Test the minimal version
print("\n Testing Minimal Chain...")
minimal_result = minimal_content_chain(
    topic="Social Media Trends 2024",
    audience="small business owners", 
    tone="practical"
)

print(" Minimal chain test completed successfully!")
print(f"\nOutline preview: {minimal_result['outline'][:200]}...")
print(f"Draft preview: {minimal_result['draft'][:200]}...")